# Part 3: Bringing Workflows to Production

## Scheduling Python Jobs

In Snowflake, [Tasks](https://docs.snowflake.com/en/user-guide/tasks-intro) are used to automate and schedule jobs. Tasks can schedule the execution of a Notebook, a stored procedure, or arbitrary SQL statements. Let's use a Task to schedule a Python Stored Procedure to apply the Customer Loyalty Score UDAF on a schedule. Next, we will set up telemetry collection for the job and set up alerts.

## Creating a Python Stored Procedure

[Stored Procedures](https://docs.snowflake.com/en/developer-guide/stored-procedure/stored-procedures-overview) allow us to run *procedural* code, like loops, if/else blocks, and other patterns which are hard to do in SQL alone. 

To create a Python Stored Procedure, all we need is a Python function that accepts a Snowpark Session object as its first argument. In Snowflake we call this the "handler" for the procedure. For example, the Python function below is a bare-bones procedure handler that uses the Session object which returns an integer:

```python
from snowflake.snowpark import Session

def my_handler(sess: Session) -> int:
    n_rows = sess.table('my_table').count()
    return n_rows
```

Let's create a procedure to calculate the Customer Loyalty scores and write them to an output table. Then, we will schedule it with a Task so we can have up-to-date loyalty scores. 

In the package picker, add the package `snowflake.core`. This package is the [Python API for Snowflake](https://docs.snowflake.com/en/developer-guide/snowflake-python-api/reference/latest/index).

In [ ]:
from snowflake.snowpark import Session
from datetime import datetime
from snowflake.snowpark.functions import lit
import logging

from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
from snowflake.snowpark.types import IntegerType

# Handler for the procedure
def calc_loyalty_scores(sess: Session) -> int:
    logging.info('Starting loyalty score procedure')
    calculated_at = datetime.now()
    
    sess.sql("""
        SELECT
          CUSTOMER_ID,
          LoyaltyScoreUDAF(ORDER_TOTAL, ORDER_TS) AS loyalty_score
        FROM tasty_bytes_orders
        WHERE 
            ORDER_TOTAL IS NOT NULL 
            AND ORDER_TS IS NOT NULL
            AND CUSTOMER_ID IS NOT NULL
        GROUP BY CUSTOMER_ID
        ORDER BY loyalty_score DESC
    """).with_column("calculated_at", lit(calculated_at))\
    .write\
    .save_as_table('loyalty_scores', mode='append')

    # Returns the number of rows in the output table
    return sess.table('tasty_bytes_orders').count()


# Register the handler as a Stored Procedure
loyalty_sproc = session.sproc.register(
    calc_loyalty_scores,
    return_type=IntegerType(),
    name='calculate_loyalty_scores',
    is_permanent=True,
    packages=['snowflake-telemetry-python'],
    stage_location='@my_stage/',
    replace=True
)

print(f'Created procedure "{loyalty_sproc.name}"')

In [ ]:
-- Call the stored proc for a test run
CALL CALCULATE_LOYALTY_SCORES();

If we click **Monitoring** > **Query History** from the table of contents on the left, we should be able to see the `CALL CALCULATE_LOYALTY_SCORES();` query in the list. (Tip: hold Ctrl as we click **Query History** to open it in a new tab.)

Now let's create a Task to run this stored procedure on a schedule. Check out the docs on [`CREATE TASK`](https://docs.snowflake.com/en/sql-reference/sql/create-task) for more information, but at its core a Task needs a **schedule** and a **warehouse**. For example, the SQL below defines a Task that runs every hour with the Warehouse, `my_wh`, and calls a stored procedure named `my_stored_procedure`.

```sql
USE DATABASE TEST_DB;
USE SCHEMA TEST_SCHEMA;

CREATE TASK my_task
  WAREHOUSE = my_wh
  SCHEDULE = '60 MINUTES'
  AS
    CALL my_stored_procedure
```

And of course, you can define the same Task using Python!

```python
from datetime import timedelta
from snowflake.core.task import Cron, Task

tasks = root.databases["TEST_DB"].schemas["TEST_SCHEMA"].tasks

task = tasks.create(
    Task(
        name="my_task",
        definition="CALL my_stored_procedure",
        schedule=Cron("0 * * * *", "America/Los_Angeles"),
        warehouse="my_wh"
    ),
)
```

In [ ]:
# Let's create a Task in Python
from snowflake.core import Root
from snowflake.core.task import Cron, Task

root = Root(session)
tasks = root.databases["HOL_DB"].schemas["PUBLIC"].tasks
schedule = Cron("*/5 * * * *", "America/Los_Angeles")  # every 5 minutes

task = tasks.create(
    Task(
        name="loyalty_score",
        definition="CALL HOL_DB.PUBLIC.CALCULATE_LOYALTY_SCORES();",
        schedule=schedule,
        warehouse="my_wh"
    ),
    mode="orreplace"
)

# Start the task, will run every 5 minutes now
task.resume()

Now the Task, `loyalty_score`, should be running _every 5 minutes_. In the Database browser on the left, find your database and under the PUBLIC schema you should see the `loyalty_score` Task listed under the *Tasks* tab.

![Task Location](https://raw.githubusercontent.com/Snowflake-Labs/sfguide-getting-started-with-data-analytics-with-sql-python/refs/heads/main/images/navigate_to_task.png) 

Click the task and it will bring you to the page for the Task. Click the **Run History** page to see the executions of the Task.

Remember, the task is running every 5th minute, so you may not see an execution in the history tab right away. You can call [`task.execute()`](https://docs.snowflake.com/en/developer-guide/snowflake-python-api/reference/latest/_autosummary/snowflake.core.task.TaskResource#snowflake.core.task.TaskResource) to force an execution of the task independent of its schedule.

In [ ]:
# Now let's stop the Task's execution
task.suspend()

## Monitoring Python Workloads

Now we have a simple Python job scheduled to execute every 5 minutes. This raises the question: how do we monitor this job as it's running? How can we be notified if it fails overnight? How can we track performance over time? Snowflake provides a [suite of observability capabilities](https://www.snowflake.com/en/product/features/snowflake-trail/) to help you address those questions. Let's go over a few of Snowflake's observability capabilities:

1. Telemetry Collection: You can route any [logs](https://docs.snowflake.com/en/developer-guide/logging-tracing/logging), [metrics](https://docs.snowflake.com/en/developer-guide/logging-tracing/metrics), or [traces](https://docs.snowflake.com/en/developer-guide/logging-tracing/tracing) from your jobs to Snowflake's Event Tables for storage and alerting.
1. History tables: Use the [Query History](https://docs.snowflake.com/en/user-guide/ui-snowsight-activity), [Copy History](https://docs.snowflake.com/en/user-guide/data-load-monitor), and [Task History](https://docs.snowflake.com/en/user-guide/ui-snowsight-tasks) to monitor all usage in your account.
1. [Alerts and Notifications](https://docs.snowflake.com/en/developer-guide/builders/observability#label-observability-alerts-notifications): Alerts allow for customizable triggering conditions, actions, and a schedule, in combination with notification integrations for proactive monitoring.
1. [Extensibility with third-party tools](https://docs.snowflake.com/en/developer-guide/builders/observability#label-observability-tools-analysis-visualization): The Snowflake [event table](https://docs.snowflake.com/en/developer-guide/logging-tracing/event-table-setting-up) adopts [OpenTelemetry](https://opentelemetry.io/docs/) standards, so your Snowflake telemetry can easily be consumed by other ecosystem tools.

## Event Table Setup

The Event Table is the central storage system for logs, metrics, and traces in your account. A [default Event Table](https://docs.snowflake.com/developer-guide/logging-tracing/event-table-setting-up#use-the-default-event-table) is already created at `snowflake.telemetry.events`. Let's enable it:

In [ ]:
ALTER ACCOUNT SET EVENT_TABLE = snowflake.telemetry.events;

Next, we'll enable the collection of logs, metrics, and traces in our account.

In [ ]:
ALTER ACCOUNT SET LOG_LEVEL = 'INFO';
ALTER ACCOUNT SET METRIC_LEVEL = 'ALL';
ALTER ACCOUNT SET TRACE_LEVEL = 'ALWAYS';

We're now collecting logs, metrics, and traces for all jobs in our account. To test this, let's run our Task again. 

In [ ]:
task.execute()

After running this procedure, go to **Monitoring** > **Traces and Logs** > **Log Explorer**. You should see the log message "`Starting loyalty score procedure`" from the procedure in the **Log Explorer**. You can use the filters at the top to find the log message. Look under the Object column for the name of the procedure, "`CALCULATE_LOYALTY_SCORES()`". (Tip: you can open the **Monitoring** page in a new browser tab by holding Ctrl as you click the tab.)

![](https://raw.githubusercontent.com/Snowflake-Labs/sfguide-getting-started-with-data-analytics-with-sql-python/refs/heads/main/images/log_explorer.png)

Click that log line and a pane will open on the right with more information about the log entry, like the object which emitted it, the warehouse used, and the file and line where the log line was emitted from. 

Click the button to go the Query associated with the log.

![](https://raw.githubusercontent.com/Snowflake-Labs/sfguide-getting-started-with-data-analytics-with-sql-python/refs/heads/main/images/link_to_query_page.png)


In the page for the Query, the "Query Telemetry" tab will show the trace diagram for the job. In this page, each row is a **span** representing a unit of work. In this case, there are spans for the Stored Procedure execution, the DataFrame queries, and the UDAF. 

![](https://raw.githubusercontent.com/Snowflake-Labs/sfguide-getting-started-with-data-analytics-with-sql-python/refs/heads/main/images/trace_diagram.png)

The concept of [spans and traces](https://opentelemetry.io/docs/concepts/observability-primer/#distributed-traces) come from [OpenTelemetry](https://opentelemetry.io/), a project in the Cloud Native Compute Foundation. OpenTelemetry is often used in observability for application infrastructure, and Snowflake has applied it to native primitives such as Stored Procedures, UDFs, DataFrames, and soon to Queries, Tasks, Notebooks, Snowpark Container Services, and more!

## Custom Telemetry

### Logs

Snowflake will collect any logs emitted by our Python code, even logs from packages we're calling. If we want to add more logs to our jobs so we can monitor and debug better in the future, we can just run `import logging` and add logging statements like `logging.warn("This is a warning message")`. [More information](https://docs.snowflake.com/en/developer-guide/logging-tracing/logging-python#stored-procedure-example).

### Span Events

We can also add custom spans using [`telemetry.set_span_attribute()`](https://docs.snowflake.com/en/developer-guide/logging-tracing/tracing-python#adding-span-attributes), these are similar to logs but we can be more useful for attaching structured data like lists and dictionaries. This data will show in the "Span Events" tab of the Trace Diagram.

## Next Steps

### Create email notifications 

Use logs, metrics, and traces to create an alert if a job throws an error log or if a Task fails. This is a great way to stay on top of scheduled data pipelines. To do so, use Snowflake's [alerts](http://docs.snowflake.com/en/user-guide/alerts) feature.

- [`SYSTEM$SEND_EMAIL()`](https://docs.snowflake.com/en/sql-reference/stored-procedures/system_send_email)
- [Send email notifications](https://docs.snowflake.com/en/user-guide/notifications/email-notifications)



#### Step 1: Create a Notification Integration

This is a Snowflake object that establishes a connection between Snowflake and an email server, allowing Snowflake to send emails.

In [ ]:
CREATE OR REPLACE NOTIFICATION INTEGRATION send_email_notification
    TYPE=EMAIL
    ENABLED=TRUE
    ALLOWED_RECIPIENTS=('bryan.blancomorales@snowflake.com','tdd4@njit.edu');

#### Step 2: Create an Alert to Monitor for Failures

Create an `ALERT`. This object will run on a schedule, check for a failed task, and send an email if the condition is met. This alert will query the `TASK_HISTORY` to see if the `loyalty_score` task has failed recently.

In [ ]:
CREATE OR REPLACE ALERT loyalty_score_failure_alert
    WAREHOUSE = my_wh
    SCHEDULE = '10 MINUTE'
    IF (EXISTS (
        SELECT 1
        FROM TABLE(INFORMATION_SCHEMA.TASK_HISTORY(
            SCHEDULED_TIME_RANGE_START => DATEADD('minute', -11, CURRENT_TIMESTAMP()),
            TASK_NAME => 'LOYALTY_SCORE'
        ))
        WHERE STATE = 'FAILED'
    ))
    THEN
        CALL SYSTEM$SEND_EMAIL(
            'send_email_notification',
            'tdd4@njit.edu',
            'Snowflake Task Failure: LOYALTY_SCORE',
            'The task named "LOYALTY_SCORE" in HOL_DB.PUBLIC has failed. Check the run history for details.'
        );

#### Step 3: Activate the Alert

Alerts are created in a suspended state by default. We must resume the alert to activate it.

In [ ]:
ALTER ALERT loyalty_score_failure_alert RESUME;

In [ ]:
DESCRIBE ALERT loyalty_score_failure_alert;

#### Step 4: Create a Failed Condition

Manually modify the Python function `calc_loyalty_scores` to query a table that does not exist, which will guarantee a SQL error. The code below changes `tasty_bytes_orders` to `tasty_bytes_orders_FAKE_TABLE`.

In [ ]:
from snowflake.snowpark import Session
from datetime import datetime
from snowflake.snowpark.functions import lit
import logging

from snowflake.snowpark.types import IntegerType

# Handler for the procedure with an intentional error
def calc_loyalty_scores_fail(sess: Session) -> int:
    logging.info('Starting loyalty score procedure (INTENTIONAL FAIL)')
    calculated_at = datetime.now()

    # This will fail because the table does not exist
    sess.sql("""
        SELECT
          CUSTOMER_ID,
          LoyaltyScoreUDAF(ORDER_TOTAL, ORDER_TS) AS loyalty_score
        FROM tasty_bytes_orders_FAKE_TABLE
        WHERE
            ORDER_TOTAL IS NOT NULL
            AND ORDER_TS IS NOT NULL
            AND CUSTOMER_ID IS NOT NULL
        GROUP BY CUSTOMER_ID
        ORDER BY loyalty_score DESC
    """).with_column("calculated_at", lit(calculated_at)).write.save_as_table('loyalty_scores', mode='append')

    return sess.table('tasty_bytes_orders').count()

# Register the faulty handler, replacing the original Stored Procedure
session.sproc.register(
    calc_loyalty_scores_fail,
    return_type=IntegerType(),
    name='calculate_loyalty_scores', # Note: We use the original name to overwrite it
    is_permanent=True,
    packages=['snowflake-telemetry-python'],
    stage_location='@my_stage/',
    replace=True
)

print('Faulty procedure "calculate_loyalty_scores" has been deployed.')

Now, trigger the task manually to run the faulty stored procedure.

In [ ]:
# Re-fetch the task object if needed
from snowflake.core import Root
root = Root(session)
task = root.databases["HOL_DB"].schemas["PUBLIC"].tasks["loyalty_score"]

task.execute()
print("Task execution triggered. It should now fail.")

#### RESULT

_**Run History tab**_

![task_run_history](https://i.imgur.com/tQMfVTE.png)

_**Email Notification**_

![email](https://i.imgur.com/Dno6QMU.png)
